<a href="https://colab.research.google.com/github/parkrye/Python/blob/main/202210_Bigdata/04_Cache%2C_Persist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster("local").setAppName("restaurant-review-average")
sc   = SparkContext(conf=conf)

In [ ]:
directory = "C:\\Users\\mhso_lec\\study_notebook\\data"
filename  = "restaurant_reviews.csv"

In [ ]:
lines = sc.textFile(f"file:///{directory}\\{filename}")

In [ ]:
lines.collect()

['id,item,cateogry,reviews,',
 '0,짜장면,중식,125,',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32,',
 '3,떡볶이,분식,534,',
 '4,라멘,일식,223,',
 '5,돈가스,일식,52,',
 '6,우동,일식,12,',
 '7,쌀국수,아시안,312,',
 '8,햄버거,패스트푸드,12,',
 '9,치킨,패스트푸드,23']

In [ ]:
header = lines.first()
filtered_lines = lines.filter(lambda row : row != header)
filtered_lines.collect()

['0,짜장면,중식,125,',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32,',
 '3,떡볶이,분식,534,',
 '4,라멘,일식,223,',
 '5,돈가스,일식,52,',
 '6,우동,일식,12,',
 '7,쌀국수,아시안,312,',
 '8,햄버거,패스트푸드,12,',
 '9,치킨,패스트푸드,23']

In [ ]:
def parse(row):
    
    fields = row.split(",")
    category = fields[2]
    reviews  = int(fields[3])
    
    return category, reviews

# `persist`를 사용하지 않는 경우

In [ ]:
# transformations를 수행할 RDD 생성
categoryReviews = filtered_lines.map(parse)
categoryReviews.collect()

[('중식', 125),
 ('중식', 235),
 ('분식', 32),
 ('분식', 534),
 ('일식', 223),
 ('일식', 52),
 ('일식', 12),
 ('아시안', 312),
 ('패스트푸드', 12),
 ('패스트푸드', 23)]

In [ ]:
result1 = categoryReviews.take(10) # action을 곧바로 실행
result2 = categoryReviews.mapValues(lambda x : (x, 1)).collect()

In [ ]:
result1, result2

([('중식', 125),
  ('중식', 235),
  ('분식', 32),
  ('분식', 534),
  ('일식', 223),
  ('일식', 52),
  ('일식', 12),
  ('아시안', 312),
  ('패스트푸드', 12),
  ('패스트푸드', 23)],
 [('중식', (125, 1)),
  ('중식', (235, 1)),
  ('분식', (32, 1)),
  ('분식', (534, 1)),
  ('일식', (223, 1)),
  ('일식', (52, 1)),
  ('일식', (12, 1)),
  ('아시안', (312, 1)),
  ('패스트푸드', (12, 1)),
  ('패스트푸드', (23, 1))])

`categoryReivews`는 `result1`, `result2`를 만들면서 2번 만들어 집니다. `categoryReviews`에서 데이터를 꺼내오기만 하면 되지, 변경은 일어나지 않기 때문에 `persist`를 이용해서 `categoryReviews`를 메모리에 넣어 놓는 것이 더 좋아 보입니다.

# `persist`를 사용하는 경우

In [ ]:
categoryReviews = filtered_lines.map(parse).persist() # categoryReviews RDD는 하나만 존재하는 RDD
categoryReviews

PythonRDD[7] at RDD at PythonRDD.scala:53

In [ ]:
result1 = categoryReviews.take(10)
result2 = categoryReviews.mapValues(lambda x : (x, 1)).collect()

In [ ]:
sc.stop()